In [185]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pandas as pd
from time import sleep
import json

In [186]:
# شيل الهمزات

In [187]:
df = pd.read_csv('../data/medical_network_cleaned.csv')
df.head()

,Name,location,Phone,Main Category,Sub Category
0,مخلص مزاهرة,الدوار السابع,5820425,أطباء,الطب العام
1,عيادة ابن النفيس الطبية,الدوار السابع,5858343,أطباء,الطب العام
2,جوسانتي للرعاية الصحية,الدوار السابع,5804444,أطباء,الطب العام
3,اويس الرواشدة,الدوار السابع,0797113535,أطباء,الطب العام
4,احمد صالح ابو رمان,الدوار الثامن,0790076768,أطباء,الطب العام


In [188]:
df['location'] = df['location'].str.strip()

In [189]:
df['location'] = df['location'].str.replace('أ', 'ا').str.replace('إ', 'ا').str.replace('آ', 'ا').str.replace('ؤ', 'و').str.replace('ئ', 'ي').str.replace('ة', 'ه')

In [190]:
location = "عمان"
geolocator = Nominatim(user_agent="jordan_location_finder_v2")
geocode = RateLimiter(
    lambda x: geolocator.geocode(x, language="ar", country_codes='JO', addressdetails=True), 
    min_delay_seconds=1, 
    max_retries=3
    )
geocode(location)

Location(عمان, ناحية عمان, لواء قصبة عمان, عمان, 11118, الأردن, (31.9515694, 35.9239625, 0.0))

In [191]:
def get_region_by_location(location):
    try:
        
        # Try the location with Jordan
        full_location = f"{location}, الأردن"

        print(f'Getting result of {full_location}')
                
        result = geocode(full_location)
        
        print(f"Found: {result.address}")
        print(f"Raw data: {result.raw}")
        
        address = result.raw.get('address', {})
        print(f"Address components: {address}")
        
        # Check various fields
        governorate = (address.get('state') or 
                        address.get('county') or 
                        address.get('city') or 
                        address.get('town') or
                        address.get('province') or
                        'Unknown')
        
        return governorate
        
    except Exception as e:
        print(f"Error: {e}")
        return 'Unknown'

In [192]:
location = pd.DataFrame(df['location'].unique())
location.rename(columns={0:'Location'}, inplace=True)
location

,Location
0,الدوار السابع
1,الدوار الثامن
2,الدوار الثالث
3,الجندويل
4,عبدون
...,...
268,السلط-الميدان
269,جبل عمان-الدوارالثالث
270,دوار الواحه
271,عيادات م. التخصصي


In [193]:
location['Region'] = location['Location'].apply(get_region_by_location)

Getting result of الدوار السابع, الأردن
Found: الدوار السابع, شارع زهران, الصويفية, ناحية وادي السير, لواء وادي السير, عمان, 11141, الأردن
Raw data: {'place_id': 43237497, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'way', 'osm_id': 404199205, 'lat': '31.9593987', 'lon': '35.8575522', 'class': 'junction', 'type': 'yes', 'place_rank': 30, 'importance': 7.10040436407571e-05, 'addresstype': 'junction', 'name': 'الدوار السابع', 'display_name': 'الدوار السابع, شارع زهران, الصويفية, ناحية وادي السير, لواء وادي السير, عمان, 11141, الأردن', 'address': {'junction': 'الدوار السابع', 'road': 'شارع زهران', 'neighbourhood': 'الصويفية', 'county': 'ناحية وادي السير', 'state_district': 'لواء وادي السير', 'state': 'عمان', 'ISO3166-2-lvl4': 'JO-AM', 'postcode': '11141', 'country': 'الأردن', 'country_code': 'jo'}, 'boundingbox': ['31.9586627', '31.9600018', '35.8566596', '35.8584568']}
Address components: {'junction': 'الدوار السابع', 'road': 'شارع زهرا

In [194]:
location[location['Region'] == 'Unknown']

,Location,Region
6,الصويفيه,Unknown
7,شارع مكه,Unknown
8,بيادر وادي السير,Unknown
12,جبل النزهه,Unknown
14,ضاحيه الاقصى,Unknown
...,...,...
266,الفحيص - الحمر,Unknown
267,السلط -الدبابنه,Unknown
269,جبل عمان-الدوارالثالث,Unknown
270,دوار الواحه,Unknown


In [195]:
display(location['Region'].unique())

array(['عمان', 'Unknown', 'البلقاء', 'الزرقاء', 'عجلون', 'المفرق',
       'محافظة جرش', 'إربد', 'الكرك', 'مادبا', 'معان'], dtype=object)

In [196]:
len(location[location['Region']=='عمان']['Location'].unique())

60

In [197]:
location[location['Region']=='Unknown']

,Location,Region
6,الصويفيه,Unknown
7,شارع مكه,Unknown
8,بيادر وادي السير,Unknown
12,جبل النزهه,Unknown
14,ضاحيه الاقصى,Unknown
...,...,...
266,الفحيص - الحمر,Unknown
267,السلط -الدبابنه,Unknown
269,جبل عمان-الدوارالثالث,Unknown
270,دوار الواحه,Unknown


In [198]:
location.to_csv('../data/locations_not_full.csv', index=False)

#### fix locations 

In [250]:
filled_location = pd.read_csv('../data/locations.csv')
filled_location['Location'] = filled_location['Location'].str.strip()
filled_location['Location'] = filled_location['Location'].str.replace('أ', 'ا').str.replace('إ', 'ا').str.replace('آ', 'ا').str.replace('ؤ', 'و').str.replace('ئ', 'ي').str.replace('ة', 'ه')
# after cleaning there are duplicates so we need to drop them
filled_location = filled_location.drop_duplicates(subset=['Location'])

normal_location = pd.read_csv('../data/locations_not_full.csv')

In [251]:
combined_locations = None
combined_locations = pd.merge(
    normal_location, 
    filled_location,
    on='Location', 
    how='inner', 
    suffixes=('_normal', '_filled')
    )[['Location', 'Region_normal', 'Region_filled']]

In [253]:
combined_locations

,Location,Region_normal,Region_filled
0,الدوار السابع,عمان,عمان
1,الدوار الثامن,عمان,عمان
2,الدوار الثالث,عمان,عمان
3,الجندويل,عمان,عمان
4,عبدون,عمان,عمان
...,...,...,...
268,السلط-الميدان,البلقاء,البلقاء
269,جبل عمان-الدوارالثالث,Unknown,عمان
270,دوار الواحه,Unknown,عمان
271,عيادات م. التخصصي,Unknown,عمان


In [254]:
# filling the region_normal with region_filled where they are unknown 
for idx, row in combined_locations.iterrows():
    if row['Region_normal'] == 'Unknown':
        combined_locations.at[idx, 'Region_normal'] = row['Region_filled']

In [255]:
combined_locations[combined_locations['Region_normal']=='غزة']

,Location,Region_normal,Region_filled
8,بيادر وادي السير,غزة,غزة
264,بيادر,غزة,غزة


In [256]:
combined_locations.loc[combined_locations['Region_normal']=='غزة', 'Region_normal'] = 'عمان'
combined_locations.loc[combined_locations['Region_normal']=='محافظة بغداد', 'Region_normal'] = 'عمان'
combined_locations.loc[combined_locations['Region_normal']=='محافظة دمشق', 'Region_normal'] = 'عمان'
combined_locations.loc[combined_locations['Region_normal']=='يهودا والسامرة', 'Region_normal'] = 'عمان'

In [273]:
combined_locations.loc[combined_locations['Location']=='شارع مكه', 'Region_normal'] = 'عمان'
combined_locations.loc[combined_locations['Location']=='خريبه السوق', 'Region_normal'] = 'عمان'
combined_locations.loc[combined_locations['Location']=='جبل النزهه', 'Region_normal'] = 'عمان'
combined_locations.loc[combined_locations['Location']=='ضاحيه الاقصى', 'Region_normal'] = 'عمان'
combined_locations.loc[combined_locations['Location']=='المدينه الرياضيه', 'Region_normal'] = 'عمان'
combined_locations.loc[combined_locations['Location']=='وسط البلد', 'Region_normal'] = 'عمان'
combined_locations.loc[combined_locations['Location']=='شارع وصفي التل', 'Region_normal'] = 'عمان'
combined_locations.loc[combined_locations['Location']=='الجبل الاخضر', 'Region_normal'] = 'عمان'
combined_locations.loc[combined_locations['Location']=='جبل الزهور', 'Region_normal'] = 'عمان'
combined_locations.loc[combined_locations['Location']=='الرابيه', 'Region_normal'] = 'عمان'
combined_locations.loc[combined_locations['Location']=='الاشرفيه', 'Region_normal'] = 'عمان'
combined_locations.loc[combined_locations['Location']=='شارع الملكه رانيا', 'Region_normal'] = 'عمان'
combined_locations.loc[combined_locations['Location']=='ضاحيه الامير راشد', 'Region_normal'] = 'عمان'
combined_locations.loc[combined_locations['Location']=='ضاحيه الحاج حسن', 'Region_normal'] = 'عمان'
combined_locations.loc[combined_locations['Location']=='مجمع خلدا بلازا', 'Region_normal'] = 'عمان'
combined_locations.loc[combined_locations['Location']=='جبل عمان- الدوار الثالث', 'Region_normal'] = 'عمان'
combined_locations.loc[combined_locations['Location']=='جبل عمان-الدوار الاول', 'Region_normal'] = 'عمان'
combined_locations.loc[combined_locations['Location']=='شارع مكه - خلف مجمع جبر', 'Region_normal'] = 'عمان'
combined_locations.loc[combined_locations['Location']=='ضاحيه الامير راشد', 'Region_normal'] = 'عمان'
combined_locations.loc[combined_locations['Location']=='الشونه الجنوبيه - الكرامه', 'Region_normal'] = 'عمان'
combined_locations.loc[combined_locations['Location']=='حي الصحابه', 'Region_normal'] = 'عمان'
combined_locations.loc[combined_locations['Location']=='ام الدنانير', 'Region_normal'] = 'عمان'
combined_locations.loc[combined_locations['Location']=='ابونصير', 'Region_normal'] = 'عمان'
combined_locations.loc[combined_locations['Location']=='شارع الجامعه الاردنيه', 'Region_normal'] = 'عمان'
combined_locations.loc[combined_locations['Location']=='جبيهه', 'Region_normal'] = 'عمان'
combined_locations.loc[combined_locations['Location']=='الجاردينز', 'Region_normal'] = 'عمان'
combined_locations.loc[combined_locations['Location']=='المناره', 'Region_normal'] = 'عمان'

In [ ]:
combined_locations.loc[combined_locations['Location']=='الطيبه', 'Region_normal'] = 'إربد'
combined_locations.loc[combined_locations['Location']=='الهاشميه', 'Region_normal'] = 'الزرقاء'
combined_locations.loc[combined_locations['Location']=='معان-وادي موسى', 'Region_normal'] = 'معان'
combined_locations.loc[combined_locations['Location']=='الفحيص - الحمر', 'Region_normal'] = 'البلقاء'
combined_locations.loc[combined_locations['Location']=='العقبه', 'Region_normal'] = 'العقبة'
combined_locations.loc[combined_locations['Location']=='الكرك - الربه', 'Region_normal'] = 'الكرك'

In [292]:
df[df['location']=='عرجان']

,Name,location,Phone,Main Category,Sub Category
2474,محمد عمر حسن سمعان,عرجان,5677782,أسنان,طب الاسنان
2475,احمد محمد خالد الجعفري,عرجان,0787755722,أسنان,طب الاسنان
2999,صيدلية القصوى,عرجان,5650016,صيدليات,الصـيدليـات المعتمـدة
3000,صيدلية سدر,عرجان,5685525,صيدليات,الصـيدليـات المعتمـدة
3001,صيدلية مركز الدواء,عرجان,5161008,صيدليات,الصـيدليـات المعتمـدة
3002,صيدلية كارين,عرجان,0795604239,صيدليات,الصـيدليـات المعتمـدة
3003,صيدلية كارمينا,عرجان,5677166,صيدليات,الصـيدليـات المعتمـدة
3004,صيدلية يوسف و جاد,عرجان,0799997620,صيدليات,الصـيدليـات المعتمـدة
3005,الصيدلية العضوية,عرجان,5692626,صيدليات,الصـيدليـات المعتمـدة
3006,صيدلية طلة عرجان,عرجان,5669007,صيدليات,الصـيدليـات المعتمـدة


In [ ]:
# الرابية
# df[df['location']=='الأشرفية']
# شارع الملكة رانيا	
# ضاحية الامير راشد	 
# ضاحية الحاج حسن	
# مجمع خلدا بلازا	
# جبل عمان- الدوار الثالث	
# جبل عمان-الدوار الأول	
# شارع مكة - خلف مجمع جبر	
# ضاحية الأمير راشد	
# الشونة الجنوبية - الكرامة	
# حي الصحابة	
# ضاحية الأقصى	
# أم الدنانير	
# ابونصير
# معان-وادي موسى	
# شارع الجامعة الاردنية	
# جبيهة
# الجاردينز
# الفحيص - الحمر	
# الكرك - الربه	

In [295]:
# combined_locations[['Location', 'Region_normal']].head(50)
# combined_locations[['Location', 'Region_normal']].iloc[50:100]
# combined_locations[['Location', 'Region_normal']].iloc[100:150]
# combined_locations[['Location', 'Region_normal']].iloc[150:200]
# combined_locations[['Location', 'Region_normal']].iloc[250:300]

In [285]:
# hospitals = combined_locations[combined_locations['Location'].str.contains('مستشفى') | combined_locations['Location'].str.contains('عيادات')]
# hospitals